In [ ]:
import pandas as pd
import geojson
from geojson import Feature, Point, FeatureCollection

# Read the house and station CSV files
houses_df = pd.read_csv('../data/curated/property/property_addresses_to_lat_lng.csv')
stations_df = pd.read_csv("../data/curated/property/victoria_metro_stations.csv")

# Drop rows with missing longitude and latitude
houses_df = houses_df.dropna(subset=['longitude', 'latitude'])
stations_df = stations_df.dropna(subset=['longitude', 'latitude'])

# Function to create GeoJSON Features
def create_geojson_feature(row, feature_type='house'):
    point = Point((row['longitude'], row['latitude']))
    properties = {
        'address': row.get('address', 'unknown'),
        'type': feature_type
    }
    return Feature(geometry=point, properties=properties)

# Convert house data to GeoJSON Features
house_features = houses_df.apply(lambda row: create_geojson_feature(row, 'house'), axis=1).tolist()
house_feature_collection = FeatureCollection(house_features)

# Convert station data to GeoJSON Features
station_features = stations_df.apply(lambda row: create_geojson_feature(row, 'station'), axis=1).tolist()
station_feature_collection = FeatureCollection(station_features)

# Save house and station data as separate GeoJSON files
houses_geojson_output_path = '../data/raw/external/API/1/property_geojson.geojson'
stations_geojson_output_path = '../data/raw/external/API/1/vic_metro_stations_geojson.geojson'

with open(houses_geojson_output_path, 'w') as f:
    geojson.dump(house_feature_collection, f)

with open(stations_geojson_output_path, 'w') as f:
    geojson.dump(station_feature_collection, f)

print("GeoJSON files for houses and stations have been generated")


In [ ]:
import geojson
import openrouteservice
import pandas as pd
import time

# Initialize OpenRouteService API client
api_key = '5b3ce3597851110001cf6248dda07331ae64496fa25c4e54b59fba30'
client = openrouteservice.Client(key=api_key)

# Read GeoJSON files
with open('../data/raw/external/API/1/property_geojson.geojson', 'r') as f:
    rent_geojson = geojson.load(f)

with open('../data/raw/external/API/1/vic_metro_stations_geojson.geojson', 'r') as f:
    stations_geojson = geojson.load(f)

# Extract latitude and longitude
def extract_coordinates(features):
    coordinates = []
    for feature in features['features']:
        lon, lat = feature['geometry']['coordinates']
        address = feature['properties'].get('address', 'unknown')
        coordinates.append((lat, lon, address))
    return coordinates

rent_coordinates = extract_coordinates(rent_geojson)
station_coordinates = extract_coordinates(stations_geojson)

# Extract coordinates for houses and stations
house_coords = [(lon, lat) for lat, lon, _ in rent_coordinates]
station_coords = [(lon, lat) for lat, lon, _ in station_coordinates]

# Use OpenRouteService API to calculate distances in batches
def calculate_distance_matrix_in_batches(house_coords, station_coords, batch_size=12):
    all_shortest_distances = []
    
    for i in range(0, len(house_coords), batch_size):
        batch_houses = house_coords[i:i + batch_size]
        try:
            matrix = client.distance_matrix(
                locations=batch_houses + station_coords,
                profile='driving-car',
                sources=list(range(len(batch_houses))),
                destinations=list(range(len(batch_houses), len(batch_houses) + len(station_coords))),
                metrics=['distance'],
                units='km'
            )
            distances = matrix['distances']
            
            # Process the distances between each house and station, finding the shortest
            for j, house_distances in enumerate(distances[:len(batch_houses)]):
                # Exclude None values and find the shortest distance
                valid_distances = [d for d in house_distances if d is not None]
                if valid_distances:
                    min_distance = min(valid_distances)
                    closest_station_idx = house_distances.index(min_distance)
                    rent_address = rent_coordinates[i + j][2]
                    closest_station = station_coordinates[closest_station_idx][2]
                    all_shortest_distances.append({
                        'rent_address': rent_address,
                        'closest_station': closest_station,
                        'min_driving_distance_km': min_distance
                    })
        except Exception as e:
            print(f"Error calculating distance matrix: {e}")
        time.sleep(1)  # Prevent rate-limiting
    
    return all_shortest_distances

# Process in batches
shortest_distances = calculate_distance_matrix_in_batches(house_coords, station_coords, batch_size=12)

# Save the shortest distance results to a CSV file
shortest_distances_df = pd.DataFrame(shortest_distances)
output_path = '../data/raw/external/API/1/closest_station_distance_final_1.csv'
shortest_distances_df.to_csv(output_path, index=False)

print(f"Results have been saved to {output_path}")
